## Adrien Design Notebook
This Jupyter notebook contains an interactive environment where Adrien design features can be explored and documented. It will be synced with the most recent Adrien library build.

In [3]:
#load "Adrien.Import.fsx" //Add packages from Adrien MyGet dev feed

### Overview

Some of the design goals of Adrien are
* Use [DiffSharp](http://diffsharp.github.io/DiffSharp/) for inspiration while improving on some of the limitations. DiffSharp is written by Atılım Güneş Baydin and Barak A. Pearlmutter, two computer scientists who have published a lot on implementing [automatic differentiation](http://www.bcl.hamilton.ie/~barak/publications.html) using functional languages. However some design choices in DiffSharp should be improved, e.g DiffSharp uses different types to represent scalars, vectors, tensors and must implement a set of operations for each type, as well as operations thst involve combining two types different types:

```F#
static member (+) (a:DV, b:DM) =
        let inline ff(a, b) = Backend.Add_V_MCols(a, b)
        let inline fd(a, b) = a + b
        let inline df_da(cp, ap, at) = DM.OfCols(b.Cols, at)
        let inline df_db(cp, bp, bt) = bt
        let inline df_dab(cp, ap, at, bp, bt) = at + bt
        let inline r_d_d(a, b) = Add_DMCols_DV(b, a)
        let inline r_d_c(a, b) = Add_DMColsCons_DV(a)
        let inline r_c_d(a, b) = Add_DMCols_DVCons(b)
        DM.Op_DV_DM_DM (a, b, ff, fd, df_da, df_db, df_dab, r_d_d, r_d_c, r_c_d)

    static member (+) (a:DM, b:DV) =
        let inline ff(a, b) = Backend.Add_V_MCols(b, a)
        let inline fd(a, b) = a + b
        let inline df_da(cp, ap, at) = at
        let inline df_db(cp, bp, bt) = DM.OfCols(a.Cols, bt)
        let inline df_dab(cp, ap, at, bp, bt) = at + bt
        let inline r_d_d(a, b) = Add_DMCols_DV(a, b)
        let inline r_d_c(a, b) = Add_DMCols_DVCons(a)
        let inline r_c_d(a, b) = Add_DMColsCons_DV(b)
DM.Op_DM_DV_DM (a, b, ff, fd, df_da, df_db, df_dab, r_d_d, r_d_c, r_c_d)
```

This design choice has significantly increased the size of the DiffSharp code base and made the code harder to read.

Adrien uses a single [type](https://github.com/Lokad/Adrien/blob/master/src/spikes/1/Adrien.Core/Numeric.fs) called `Numeric` to represent numeric values and operations.  These operations are represented as computation trees and need not be actually computed until evaluation time when the configured Backend will interpret the tree and emit the correct IL code to handle the different `Numeric` shapes and formats.

In [4]:
open Adrien.Numeric
let c1 = scalar 4.40f
c1

{Shape = Scalar;
 Format = Float32;
 Data = 4.4000001f;
 Op = None;
 Left = None;
 Right = None;}

In [5]:
let c2 = sin c1
c2

{Shape = Scalar;
 Format = Float32;
 Data = null;
 Op = Some Sin;
 Left = Some {Shape = Scalar;
              Format = Float32;
              Data = 4.4000001f;
              Op = None;
              Left = None;
              Right = None;};
 Right = None;}

In [6]:
let v1 = vector [|1.0f;2.0f|]
v1

{Shape = Vector;
 Format = Float32;
 Data = [|1.0f; 2.0f|];
 Op = None;
 Left = None;
 Right = None;}

In [7]:
let r = v1 * c1
r

{Shape = Vector;
 Format = Float32;
 Data = null;
 Op = Some Mul;
 Left = Some {Shape = Vector;
              Format = Float32;
              Data = [|1.0f; 2.0f|];
              Op = None;
              Left = None;
              Right = None;};
 Right = Some {Shape = Scalar;
               Format = Float32;
               Data = 4.4000001f;
               Op = None;
               Left = None;
               Right = None;};}

### Design Goal
* Exploit the functional nature of F# as much as possible. So auto-differentiable mathematical functions should be defined using the same syntax as any other F# functions, e.g.:

In [8]:
open Adrien.Expression
open Adrien.Tree

let f x = sin x + constant 2.0f
f

<fun:it@5> : (Expression -> Expression)

The function f is a regular F# function. The special `Expression` type is inferred from the use of the `constant` function to declare a constant. This function can be automatically converted into a computation `Tree` consisting of primitive numeric operations.

In [8]:
tree f

Tree {Parent = 0;
      Id = 1;
      Op = Add;
      Left = OperatorNode {Parent = 1;
                           Id = 2;
                           Op = Sin;
                           Left = ValueNode {Id = 3;
                                             Shape = Symbol;
                                             Format = Nan;
                                             Parent = 2;
                                             Data = "x1";};
                           Right = None;};
      Right = Some (ValueNode {Id = 4;
                               Shape = Scalar;
                               Format = Float32;
                               Parent = 1;
                               Data = 2.0f;});}

The Numeric type provides for both numeric values and symbolic values like variables that are to be treated symbolically during computation.

In [10]:
open Adrien.Expression
open Adrien.Tree
let g y (x:Expression) = sin x + cos y
g

<fun:it@4-1> : (Expression -> (Expression -> Expression))

Functions of more than one variable can be defined and are handled in the same manner. The different independent variables are labelled x1, x2...

In [5]:
tree g

Tree {Parent = 0;
      Id = 1;
      Op = Add;
      Left = OperatorNode {Parent = 1;
                           Id = 2;
                           Op = Sin;
                           Left = ValueNode {Id = 3;
                                             Shape = Symbol;
                                             Format = Nan;
                                             Parent = 2;
                                             Data = "x1";};
                           Right = None;};
      Right = Some (OperatorNode {Parent = 1;
                                  Id = 4;
                                  Op = Cos;
                                  Left = ValueNode {Id = 3;
                                                    Shape = Symbol;
                                                    Format = Nan;
                                                    Parent = 2;
                                                    Data = "x2";};
                                  Right = None;});}

 Function composition is also supported:

In [11]:
let h  = cos >> g
h

<fun:h@1> : (Expression -> (Expression -> Expression))

The independent variables are handled correctly when the tree is constructed.

In [12]:
tree h

Tree
  {Parent = 0;
   Id = 1;
   Op = Add;
   Left = OperatorNode {Parent = 1;
                        Id = 2;
                        Op = Sin;
                        Left = ValueNode {Id = 3;
                                          Shape = Symbol;
                                          Format = Nan;
                                          Parent = 2;
                                          Data = "x1";};
                        Right = None;};
   Right =
    Some (OperatorNode {Parent = 1;
                        Id = 4;
                        Op = Cos;
                        Left = OperatorNode {Parent = 2;
                                             Id = 3;
                                             Op = Cos;
                                             Left = ValueNode {Id = 4;
                                                               Shape = Symbol;
                                                               Format = Nan;
                                  